In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(logits)
print(pred_probab)
print(f"Predicted class: {y_pred}")

tensor([[ 0.0784, -0.0592,  0.0096, -0.0183, -0.0862,  0.0338, -0.0796, -0.0003,
          0.0844,  0.0394]], device='mps:0', grad_fn=<LinearBackward0>)
tensor([[0.1080, 0.0941, 0.1008, 0.0980, 0.0916, 0.1032, 0.0922, 0.0998, 0.1086,
         0.1038]], device='mps:0', grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([8], device='mps:0')


In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-3.4447e-01, -1.9040e-01,  2.5145e-01, -2.5380e-01, -2.6076e-01,
          7.8926e-02, -2.6717e-01, -7.8339e-02,  1.3778e-01, -3.3173e-02,
          1.7741e-01,  5.5225e-01, -2.8982e-01,  2.5154e-01, -2.4098e-01,
          1.0295e-01, -5.7112e-01,  2.1872e-01,  1.7547e-01,  8.8740e-02],
        [-4.9672e-01, -4.3867e-01,  1.0006e-01, -2.1002e-03, -1.2588e-02,
         -2.3247e-02,  3.8193e-01, -1.0393e-04, -1.4847e-01, -9.7228e-03,
          8.6856e-02,  4.7727e-01, -6.2506e-01,  8.4611e-01, -1.9049e-01,
          1.2893e-01, -5.4740e-01, -3.9789e-02,  3.2615e-01,  1.2303e-01],
        [-3.7310e-01, -1.7993e-01,  3.2138e-01, -1.5118e-01, -2.5265e-01,
          4.6497e-02,  2.3845e-03,  1.9014e-02, -4.6973e-02, -7.7128e-01,
          5.0340e-01,  2.5934e-01, -3.0955e-01,  5.0811e-01,  2.4145e-01,
         -7.7470e-03, -5.0956e-01,  1.0559e-01,  1.5849e-01,  8.4454e-02]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.2515, 0.0000, 0.0000,

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0082, -0.0180, -0.0228,  ...,  0.0080,  0.0237, -0.0262],
        [-0.0138, -0.0302,  0.0182,  ...,  0.0287, -0.0043,  0.0093]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0127,  0.0038], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0097,  0.0138,  0.0283,  ...,  0.0082,  0.0135,  0.0435],
        [ 0.0277,  0.0229,  0.0222,  ...,  0.0249,  0.0395, -0.0257]],
       device='mps:0', grad_fn=<Slice